In [ ]:
# Script to run SISTR: 
# Method to obtain posterior estimate of s and corresponding p value for each STR

### Imports ###

import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib import pyplot as plt
from scipy.stats import geom
import copy
import sys
sys.path.append("/storage/BonnieH/selection_project/helper_functions")
from LRT_functions import *

In [ ]:
### Main function ###
def main():
    # Load parameters
    constant_het = 0.005
    denom_het = 3
    constant_common = 1
    denom_common = 1
    eps_bins = 0.3
    inFile = '/storage/BonnieH/selection_project/ssc_files/0810/DMPK.txt'
    use_het = 'y'
    use_common = 'n'
    use_bins = 'y'
    num_bins = 5
    abc_model = 'eurodem_prior2' 
    lrt_model = 'eurodem_0810'
    LRT_num_sims = 2000
    
    
    allele_freqs_file = open(inFile, 'r')
   
    print("chrom" + "\t" + "start" + "\t" + "end" + "\t" + "total" + "\t" + "period" + \
                  "\t" + "optimal_ru" + "\t" + "motif" + "\t" + "coding" + "\t"  + 'intron' + \
                  '\t' + 'UTR5' + '\t' + 'UTR3' + '\t' + 'promoter5kb' + '\t' + 'intergenic' + \
                  "\t" + "tss_gene" + "\t" + "het" + "\t" + "common" + "\t" + "bins" + "\t" + \
                  "ABC_s_median" + "\t" + "ABC_s_95%_CI" + "\t" + "Num_s_accepted_(max_10000)" + "\t" + \
                  "Likelihood_0" + "\t" + "Likelihood_s" + "\t" + "LR" + "\t" + "LogLR" + \
                  "\t" + "LRT_p_value" + "\n")

    total_lines_acc = 0 # Total lines with >10 s accepted
    total_lines = 0 
    lower_0 = 0 # Total lines where lower bound is 0
    s_acc = [] # List of percent of s accepted
     
    # Preprocess ABC lookup table
    # Get ABC tables
    ABC_tables = {}
    #opt_allele_dic = {}
    #opt_allele_dic[3] = [5,6,7,8,9,10,11,12,13]
    #opt_allele_dic[2] = [11,12,13,14,15,16,17,18,19,20]
    #opt_allele_dic[4] = [7,8,9,10]
    
    opt_allele_dic_w_per = {}
    opt_allele_dic_w_per[1] = [(1,12),(1,16),(1,20),(1,24),(1,28),(1,32),(1,36),(1,40),(1,44)]
    opt_allele_dic_w_per[3] = [(3,5),(3,6),(3,7),(3,8),(3,9),(3,10),(3,11),(3,12),(3,13)]
    opt_allele_dic_w_per[2] = [(2,11),(2,12),(2,13),(2,14),(2,15),(2,16),(2,17),(2,18),(2,19),(2,20)]
    opt_allele_dic_w_per[4] = [(4,7),(4,8),(4,9),(4,10)]
    
    pers = [1,2,3,4]
    for per in pers:
        for per_opt in opt_allele_dic_w_per[per]:
            period = per_opt[0]
            opt_allele = per_opt[1]
            file = '/gymreklab-tscc/bonnieh/abc/results/' + abc_model + '/' + str(period) + '_' + str(opt_allele) + '.txt' 
            table = GetABCList(file, num_bins)

            ABC_tables[per_opt] = table
        
        
    # Perform ABC on each locus
    for line in allele_freqs_file:
        # Get information from line
        total_lines = total_lines + 1
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        freqs = info[3]
        total = info[4]
        per = int(info[5])
        motif = info[6]
        coding = info[7]
        intron = info[8]
        UTR5 = info[9]
        UTR3 = info[10]
        promoter5kb = info[11]
        intergenic = info[12]
        gene = info[13]

        list_num = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3483709273182957, 0.00046992481203007516, 0.0010964912280701754, 0.006422305764411027, 0.0015664160401002505, 0.013157894736842105, 0.12844611528822056, 0.14364035087719298, 0.1787280701754386, 0.06531954887218046, 0.009241854636591478, 0.008771929824561403, 0.005325814536340852, 0.0012531328320802004, 0.0043859649122807015, 0.01331453634085213, 0.024592731829573933, 0.01237468671679198, 0.007832080200501253, 0.006735588972431077, 0.004855889724310777, 0.0045426065162907266, 0.002819548872180451, 0.0021929824561403508, 0.002349624060150376, 0.0009398496240601503, 0.00046992481203007516, 0.0006265664160401002, 0, 0, 0, 0, 0.00015664160401002505]
        count_num = 0
        for elem in list_num:
            count_num = count_num + elem
        print(count_num)
        list_num_copy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3483709273182957, 0.00046992481203007516, 0.0010964912280701754, 0.006422305764411027, 0.0015664160401002505, 0.013157894736842105, 0.12844611528822056, 0.14364035087719298, 0.1787280701754386, 0.06531954887218046, 0.009241854636591478, 0.008771929824561403, 0.005325814536340852, 0.0012531328320802004, 0.0043859649122807015, 0.01331453634085213, 0.024592731829573933, 0.01237468671679198, 0.007832080200501253, 0.006735588972431077, 0.004855889724310777, 0.0045426065162907266, 0.002819548872180451, 0.0021929824561403508, 0.002349624060150376, 0.0009398496240601503, 0.00046992481203007516, 0.0006265664160401002, 0, 0, 0, 0, 0.00015664160401002505]
        if list_num != list_num_copy:
            print('False')
        else: 
            print('True')
        opt_allele_dic = {}
        
        opt_allele_dic[5] = int(0.3483709273182957*10000)
        opt_allele_dic[6] = int(0.00046992481203007516*10000)
        opt_allele_dic[7] = int(0.0010964912280701754*10000)
        opt_allele_dic[8] = int(0.006422305764411027*10000)
        opt_allele_dic[9] = int(0.0015664160401002505*10000)
        opt_allele_dic[10] = int(0.013157894736842105*10000)
        opt_allele_dic[11] = int(0.12844611528822056*10000)
        opt_allele_dic[12] = int(0.14364035087719298*10000)
        opt_allele_dic[13] = int(0.1787280701754386*10000)
        opt_allele_list = range(5, 14, 1)
        for opt_allele in opt_allele_list:
            
        # Get optimal allele and allele_freqs
        
            opt_allele, allele_freqs = Process_Freqs_Given_Opt(freqs, per, end, start, opt_allele)
            print('Opt allele' + str(opt_allele))
            print(allele_freqs)
            #numpy.random.choice(numpy.arange(1, 7), p=[0.1, 0.05, 0.05, 0.2, 0.4, 0.2])
            freq_string = ','.join(str(round(item, 5)) for item in allele_freqs)
        
            # Add 0s to allele frequency list if number of alleles less than number of bins
            if len(allele_freqs) < num_bins:
                num_zeros_to_add = int((num_bins - len(allele_freqs))/2)
                for i in range(0, num_zeros_to_add):
                    freq_string = '0.0,' + freq_string
                    freq_string = freq_string + ',0.0'
                    
            # Get summary stats
            obs_het, obs_common, obs_bins = GetSummStats(freq_string, num_bins)
        
            print(chrom + '\t' + str(start) + '\t' + str(end) + '\t' + total + '\t' + str(per) + \
                      '\t' + str(opt_allele) + '\t' + motif + '\t' + coding + '\t'  + intron + \
                      '\t' + UTR5 + '\t' + UTR3 + '\t' + promoter5kb + '\t' + intergenic + \
                      '\t' + gene + '\t' + str(round(obs_het, 7)) + '\t' + str(obs_common) + '\t' + \
                      ','.join(str(round(item,4)) for item in obs_bins) + '\t')
        
            # All optimal alleles > 13 have the same mutation rate as optimal allele 13
            if per == 3 and opt_allele > 13: 
                opt_allele = 13
            if per == 3 and opt_allele < 5:
                opt_allele = 5
            
        
            # Read abcFile line by line and place in lookup table in the form of a list
        
            abc_list = ABC_tables[(per, opt_allele)]
            #abc_list = abc_list[0:opt_allele_dic[opt_allele]]
            # Perform ABC
            s_ABC, lower_bound, upper_bound, num_accepted, s_accepted = Get_S_ABC(abc_list, 
                                           obs_het, obs_common, obs_bins, constant_het, 
                                           denom_het, constant_common, denom_common, eps_bins, use_het, 
                                           use_common, use_bins)
            print('s_accepted')
            print(s_accepted)
            s_acc.extend(s_accepted)
        print('Final estimate of s')
        print(s_acc)
        '''
        # Write N/A if <10 s accepted during ABC
        if s_ABC == -1:
            s_ABC = 'N/A'
            ABC_conf_int = 'N/A'
            num_accepted = '<10'
            likelihood_0 = 'N/A'
            likelihood_s_ABC = 'N/A'
            LR = 'N/A'
            LogLR = 'N/A'
            pval = 'N/A'
        
            print(str(s_ABC) + '\t' + ABC_conf_int + '\t' + str(num_accepted) + '\t')
            print(str(likelihood_0) + '\t' + str(likelihood_s_ABC) + '\t' + \
                          str(LR) + '\t' + str(LogLR) + '\t' + str(pval) + '\n')
         
        # Perform LRT if can get posterior estimate of s
        else:
            # Round s values < 10^-5 to 0
       
            if s_ABC < 10**-5:
                s_ABC = 0
            else:
                s_ABC = round(s_ABC, 5)

            if lower_bound < 10**-5:
                lower_bound = 0
            else:
                lower_bound = round(lower_bound, 5)

            if upper_bound < 10**-5:
                upper_bound = 0
            else:
                upper_bound = round(upper_bound, 5)

            if lower_bound == 0:
                lower_0 = lower_0 + 1

            if lower_bound != -1:
                total_lines_acc = total_lines_acc + 1

            if num_accepted >= 10:
                s_acc.append(num_accepted)

            ABC_conf_int = '(' + str(lower_bound) + ' , ' + str(upper_bound) + ')'

            print(str(s_ABC) + '\t' + ABC_conf_int + '\t' + str(num_accepted) + '\t')

            s_ABC_round = get_LRT_bin(s_ABC)

            # Perform LRT

            ### Get list of s values in LRT simulations file ###
            s_list_available = []
            lrtFile = '/gymreklab-tscc/bonnieh/lrt/results/' + lrt_model + '/' \
            + str(per) + '_' + str(opt_allele) + '_freqs.txt' 
            
            lrt_file = open(lrtFile, 'r')
            header = lrt_file.readline().strip()

            for line in lrt_file:
                info = line.strip().split('\t')
                s = float(info[0])
                if s not in s_list_available:
                    s_list_available.append(s)

            # Get nearest s in LRT file to perform LRT
            if s_ABC_round not in s_list_available:
                s_ABC_round = getNearestS(s_ABC_round, s_list_available)

            # Get LRT summary statistic tables for s = 0
            
            lrtFile_for_s_0 = '/gymreklab-tscc/bonnieh/lrt/results/' + lrt_model + '/' + str(per) + '_' + str(opt_allele) + '_15_freqs.txt' 
            freqs_list_raw_0 = GetLRTListByRow(lrtFile_for_s_0, 0)
            LRT_table_0_het = []
            LRT_table_0_common = []
            LRT_table_0_bins = []

            # Get summary statistics from allele frequencies
            for freq_string in freqs_list_raw_0:

                obs_het_0, obs_common_0, obs_bins_0 = GetSummStats(freq_string, num_bins)
                LRT_table_0_het.append(obs_het_0) 
                LRT_table_0_common.append(obs_common_0) 
                LRT_table_0_bins.append(obs_bins_0)

            # Get LRT summary statistic tables for s = s_ABC_round
            if s_ABC_round == 0:
                freqs_list_raw_s = GetLRTListByRow(lrtFile_for_s_0, 1)
            else:
                freqs_list_raw_s = GetLRTListFreq(lrtFile, s_ABC_round)
            
            LRT_table_s_het = []
            LRT_table_s_common = []
            LRT_table_s_bins = []

            # Get summary statistics from allele frequencies
            for freq_string in freqs_list_raw_s:

                obs_het_s, obs_common_s, obs_bins_s = GetSummStats(freq_string, num_bins)
                LRT_table_s_het.append(obs_het_s) 
                LRT_table_s_common.append(obs_common_s) 
                LRT_table_s_bins.append(obs_bins_s)

            # Perform LRT
            likelihood_0, likelihood_s_ABC, LR, LogLR, pval = LikelihoodRatioTest(LRT_table_0_het, \
                                    LRT_table_s_het, LRT_table_0_common, LRT_table_s_common, \
                                    LRT_table_0_bins, LRT_table_s_bins, LRT_num_sims, \
                                    obs_het, obs_common, obs_bins, constant_het, denom_het, \
                                    constant_common, denom_common, eps_bins, use_het, use_common, use_bins)

            print(str(round(likelihood_0, 7)) + '\t' + str(round(likelihood_s_ABC, 7)) + '\t' + \
                              str(round(LR, 7)) + '\t' + str(round(LogLR ,7)) + '\t' + str(round(pval, 7)) + '\n')

    allele_freqs_file.close()
            
    
    # Output statistics
    print("Total lines: " + str(total_lines) + "\n")
    percent_acc = total_lines_acc/total_lines
    print("Total accepted lines: " + str(total_lines_acc) + " " + str(percent_acc) + "\n")
    percent_0 = lower_0/total_lines
    print("Lower bound for s is zero: " + str(lower_0) + " " + str(percent_0) + "\n")
    
    for i in range(0, len(s_acc)):
        s_acc[i] = s_acc[i]/10000 * 100
    
    s_acc_med = np.median(s_acc)
    s_acc_mean = np.mean(s_acc)

    print("s_acc median: " + str(s_acc_med) + "\n")
    print("s_acc mean: " + str(s_acc_mean) + "\n")
 
    plt.figure(1)
    buckets = list(np.arange(0, 101, 1)) 
    
    plt.hist(s_acc, bins=buckets, weights=np.ones(len(s_acc)) / len(s_acc))

    plt.title('Percent s accepted during ABC %s model \n const_het %.5f, denom_het = %d, num_bins = %d \n Summ stats used: het = %s common alleles = %s bins = %s'%(abc_model, constant_het, denom_het, num_bins, use_het, use_common, use_bins))
    plt.xlabel("% s accepted")
    plt.ylabel("Frequency")
    #plt.savefig(figFile, bbox_inches='tight')
    '''
        

In [ ]:
if __name__ == '__main__':
    main()